In [394]:
import pandas as pd
import numpy as np

In [428]:
df = pd.read_json('purchase_data.json')
df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


## Player Count

In [518]:
player_count = df['SN'].nunique()
player_count_df = pd.DataFrame({
    'Total Players': [player_count]
})
player_count_df

,Total Players
0,573


## Purchasing Analysis Total

In [512]:
item_count = df['Item ID'].nunique()
average_price = '${}'.format(round(df['Price'].mean(),2))
purchase_count = len(df)
total_rev = '${}'.format(round(df['Price'].sum(),2))


In [520]:
purchasing_analysis_df = pd.DataFrame({
    "Number of Unique Items": [item_count],
    "Average Price": [average_price],
    "Number of Purchases": [purchase_count],
    "Total Revenue": [total_rev]
})
purchasing_analysis_df

,Average Price,Number of Purchases,Number of Unique Items,Total Revenue
0,$2.93,780,183,$2286.33


## Gender Demographics

In [408]:
gender = df.groupby('Gender')['SN'].nunique().reset_index()
gender = gender.rename(columns={'SN': 'Gender_Count'})
total_players = gender.Gender_Count.sum()
gender['Percentage of Players'] = (gender['Gender_Count']/total_players*100).map('{:.2f}%'.format)
gender

,Gender,Gender_Count,Percentage of Players
0,Female,100,17.45%
1,Male,465,81.15%
2,Other / Non-Disclosed,8,1.40%


## Purchasing Analysis (Gender)

In [432]:
gender_purchase_analysis = df.groupby('Gender').Price.agg(['count', 'mean', 'sum']).reset_index()
gender_purchase_analysis = gender_purchase_analysis.rename(columns={'count': 'Purchase Count', 'mean': 'Avg Purchase Price','sum': 'Total Purchase Value'})
gender_purchase_analysis['Normalized'] = gender_purchase_analysis['Total Purchase Value'] / gender['Gender_Count']
gender_purchase_analysis['Avg Purchase Price'] = (gender_purchase_analysis['Avg Purchase Price']).map('${:.2f}'.format)
gender_purchase_analysis['Total Purchase Value'] = gender_purchase_analysis['Total Purchase Value'].map('${:,.2f}'.format)
gender_purchase_analysis['Normalized'] = (gender_purchase_analysis['Normalized']).map('${:.2f}'.format)
gender_purchase_analysis

,Gender,Purchase Count,Avg Purchase Price,Total Purchase Value,Normalized
0,Female,136,$2.82,$382.91,$3.83
1,Male,633,$2.95,"$1,867.68",$4.02
2,Other / Non-Disclosed,11,$3.25,$35.74,$4.47


## Age Demographics

In [523]:
bins = [0, 7, 11, 16, 21, 26, 31, 36, 41, 46]
age_labels = ['Under 7', '7-11', '12-16', '17-21', '22-26', '27-31', '32-36', '37-41', '42-46']
df['age_labels'] = pd.cut(df['Age'], bins, labels=age_labels)
age_counts = (df.groupby("age_labels"))['SN'].nunique().reset_index()
age_counts = age_counts.rename(columns={'SN': 'Total Count'})
age_counts

,age_labels,Total Count
0,Under 7,11
1,7-11,16
2,12-16,68
3,17-21,154
4,22-26,208
5,27-31,54
6,32-36,37
7,37-41,22
8,42-46,3


In [524]:
age_purchase_analysis = df.groupby('age_labels').Price.agg(['count', 'mean', 'sum']).reset_index()
age_purchase_analysis = age_purchase_analysis.rename(columns={'count': 'Purchase Count', 'mean': 'Avg Purchase Price','sum': 'Total Purchase Value'})
age_purchase_analysis['Normalized'] = age_purchase_analysis['Total Purchase Value'] / age_counts['Total Count']
age_purchase_analysis['Avg Purchase Price'] = (age_purchase_analysis['Avg Purchase Price']).map('${:.2f}'.format)
age_purchase_analysis['Total Purchase Value'] = (age_purchase_analysis['Total Purchase Value']).map('${:.2f}'.format)
age_purchase_analysis['Normalized'] = (age_purchase_analysis['Normalized']).map('${:.2f}'.format)
age_purchase_analysis

,age_labels,Purchase Count,Avg Purchase Price,Total Purchase Value,Normalized
0,Under 7,19,$2.92,$55.47,$5.04
1,7-11,22,$3.09,$67.91,$4.24
2,12-16,92,$2.81,$258.10,$3.80
3,17-21,204,$2.88,$588.40,$3.82
4,22-26,275,$2.96,$814.07,$3.91
5,27-31,79,$2.98,$235.61,$4.36
6,32-36,49,$3.08,$150.78,$4.08
7,37-41,37,$2.90,$107.35,$4.88
8,42-46,3,$2.88,$8.64,$2.88


## Top Spenders

In [526]:
purchases_by_sn = df.groupby('SN').Price.agg(['count', 'mean', 'sum']).nlargest(5, 'sum').reset_index()
purchases_by_sn = purchases_by_sn.rename(columns={'count': 'Purchase Count', 'mean': 'Avg Purchase Price','sum': 'Total Purchase Value'})
purchases_by_sn['Avg Purchase Price'] = purchases_by_sn['Avg Purchase Price'].map('${:,.2f}'.format)
purchases_by_sn['Total Purchase Value'] = purchases_by_sn['Total Purchase Value'].map('${:,.2f}'.format)
purchases_by_sn

,SN,Purchase Count,Avg Purchase Price,Total Purchase Value
0,Undirrala66,5,$3.41,$17.06
1,Saedue76,4,$3.39,$13.56
2,Mindimnya67,4,$3.18,$12.74
3,Haellysu29,3,$4.24,$12.73
4,Eoda93,3,$3.86,$11.58


## Most Popular Items

In [471]:
pop_items = df.groupby(['Item ID', 'Item Name', 'Price']).Price.agg(['count', 'sum']).nlargest(5, 'count').reset_index()
pop_items = pop_items.rename(columns={'count': 'Purchase Count', 'sum': 'Total Purchase Value'})
pop_items['Price'] = pop_items['Price'].map('${:,.2f}'.format)
pop_items['Total Purchase Value'] = pop_items['Total Purchase Value'].map('${:,.2f}'.format)
pop_items

,Item ID,Item Name,Price,Purchase Count,Total Purchase Value
0,39,"Betrayal, Whisper of Grieving Widows",$2.35,11,$25.85
1,84,Arcane Gem,$2.23,11,$24.53
2,13,Serenity,$1.49,9,$13.41
3,31,Trickster,$2.07,9,$18.63
4,34,Retribution Axe,$4.14,9,$37.26


## Most Profitable Items

In [473]:
profitable_item = df.groupby(['Item ID', 'Item Name', 'Price']).Price.agg(['count', 'sum']).nlargest(5, 'sum').reset_index()
profitable_item = profitable_item.rename(columns={'count': 'Purchase Count', 'sum': 'Total Purchase Value'})
profitable_item['Total Purchase Value'] = profitable_item['Total Purchase Value'].map('${:,.2f}'.format)
profitable_item['Price'] = profitable_item['Price'].map('${:,.2f}'.format)
profitable_item

,Item ID,Item Name,Price,Purchase Count,Total Purchase Value
0,34,Retribution Axe,$4.14,9,$37.26
1,115,Spectral Diamond Doomblade,$4.25,7,$29.75
2,32,Orenmir,$4.95,6,$29.70
3,103,Singed Scalpel,$4.87,6,$29.22
4,107,"Splitter, Foe Of Subtlety",$3.61,8,$28.88


## Three Observable Trends

1: The majority of players who are making purchases are males. They spent $1,867.68.

2: In regard to ages, the majority of players are between the ages of 17 and 26, making up 63% of the total players. This age group also spends the most, spending $1402.47 which is more than half the total revenue. 

3: No single player makes a large amount of purchases/spends a significant amount of money as compared with the rest (doesn't appear there are any major outliers); with the biggest spender being Undirrala66 who made 5 purchases. This is both the highest number of purchases and higest total spend for one user.  